In [ ]:
#————————————————————

# Name: Azure OpenAI Vanilla

# Purpose:

# This notebook will create a GPT Assistant data engineer using the OpenAI API, give it a data set, ask it to process it using it's instructions, and then prepare the resulting dataframe for download. The idea here is to test if the assistant can successfully complete data prep and some data engineering, then reduce the feature set using a logistic regression with LASSO regularization, subsetting only non-zero LASSO coefficient features in the final dataset. 

# This notebook will create a GPT Assistant using OpenAI's API and provide it with the training dataframe returned by the data engineer Assistant and a set of instructions to creating an "Extra Trees" Random Forest. Basic outline of instructions for the modeler:

# 1. Load the provided dataframe into a pandas df.
# 2. Split the data set into training and testing using a 75:25 split.
# 3. Train an Extra Trees random forest with 2000 trees.
# 4. Use the testing data to measure the model's accuracy, presicion, recall, and generate a confusion matrix.
# 5. Return the results in a single csv table. 

# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

#General Sources:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/switching-endpoints?source=recommendations

#Openai Usage:
#

#Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\azure-openai-lab\support\requirements\requirements.txt

#————————————————————

In [1]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
load_dotenv(dotenv_path=Path("C:\Python\azure-openai-lab\.venv\.env"))
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [3]:
ingredients = "Beef, Butter, Mushrooms, Onions, Cream"

In [ ]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version="2023-05-15"
        )

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   #temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": "You are a helpful cook"},
       {"role": "user", "content": "Create a flavourful recipe using the following ingredients:" + "\n" + "---" + "\n" + ingredients}
   ]
)

print("Summary: " + response.choices[0].message.content + "\n")

In [15]:
# Few-Shot learning examples

ingredients_1 = """'pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water'"""
assistant_content_1 = """{"name":"backyard style  barbecued ribs","minutes":120,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'south-west-pacific', 'main-dish', 'pork', 'oven', 'holiday-event', 'stove-top', 'hawaiian', 'spicy', 'copycat', 'independence-day', 'meat', 'pork-ribs', 'super-bowl', 'novelty', 'taste-mood', 'savory', 'sweet', 'equipment', '4-hours-or-less']","nutrition":"[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]","n_steps":10,"steps":"['in a medium saucepan combine all the ingredients for sauce#1 , bring to a full rolling boil , reduce heat to medium low and simmer for 1 hour , stirring often', 'rub the ribs with soy sauce , garlic , ginger , chili powder , pepper , salt and chopped cilantro , both sides !', 'wrap ribs in heavy duty foil', 'let stand 1 hour', 'preheat oven to 350 degrees', 'place ribs in oven for 1 hour , turning once after 30 minutes', '3 times during cooking the ribs open foil wrap and drizzle ribs with sauce#1', 'place all the ingredients for sauce#2 in a glass or plastic bowl , whisk well and set aside', 'remove ribs from oven and place on serving platter', 'offer both sauces at table to drizzle over ribs']","description":"this recipe is posted by request and was originaly from chef sam choy's cookbook ","ingredients":"['pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water']","n_ingredients":22}"""

ingredients_2 = """'lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup'"""
assistant_content_2 = """{"name":"chicken lickin  good  pork chops","minutes":500,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'main-dish', 'pork', 'crock-pot-slow-cooker', 'dietary', 'meat', 'pork-chops', 'equipment']","nutrition":"[105.7, 8.0, 0.0, 26.0, 5.0, 4.0, 3.0]","n_steps":5,"steps":"['dredge pork chops in mixture of flour , salt , dry mustard and garlic powder', 'brown in oil in a large skillet', 'place browned pork chops in a crock pot', 'add the can of soup , undiluted', 'cover and cook on low for 6-8 hours']","description":"here's and old standby i enjoy from time to time. it's from an old newspaper clipping i cut out years ago. very tasty.","ingredients":"['lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup']","n_ingredients":7}"""

ingredients_3 = """'boneless skinless chicken breast halves', 'condensed cream of chicken soup', 'egg', 'seasoning salt', 'all-purpose flour', 'cornstarch', 'garlic powder', 'paprika', 'salt and pepper', 'oil'"""
assistant_content_3 = """{"name":"crispy crunchy  chicken","minutes":35,"tags":"['60-minutes-or-less', 'time-to-make', 'course', 'preparation', 'healthy', 'main-dish', 'dietary', 'low-saturated-fat', 'low-in-something']","nutrition":"[335.8, 11.0, 2.0, 24.0, 64.0, 10.0, 10.0]","n_steps":8,"steps":"['combine soup , egg and seasoned salt in a bowl and set aside', 'mix together flour , cornstarch , garlic powder , paprika , salt and pepper in a resealable plastic bag', 'dip chicken pieces into soup mixture and turn so as to coat all over', 'place chicken pieces in bag with flour mixture , seal bag and shake to coat chicken', 'place coated pieces of chicken on a platter and allow to set until the coating becomes doughy', 'heat oil in a deep fryer or in a skillet over medium heat , using enough oil to cover chicken pieces when fried', 'once chicken is doughy , fry pieces in oil for approx 5-8 minutes or until cooked through and juices run clear', 'drain pieces on paper towel and serve']","description":"delicious, crunchy fried chicken. this recipe came from the ","ingredients":"['boneless skinless chicken breast halves', 'condensed cream of chicken soup', 'egg', 'seasoning salt', 'all-purpose flour', 'cornstarch', 'garlic powder', 'paprika', 'salt and pepper', 'oil']","n_ingredients":10}"""

ingredients_4 = """'all-purpose flour', 'granulated sugar', 'baking powder', 'salt', 'vanilla extract', 'egg', 'milk', 'vegetable oil', 'bread', 'brown sugar', 'ground cinnamon', 'butter', 'powdered sugar'"""
assistant_content_4 = """{"name":"deep fried dessert thingys","minutes":20,"tags":"['30-minutes-or-less', 'time-to-make', 'course', 'preparation', 'occasion', 'low-protein', 'desserts', 'deep-fry', 'stove-top', 'dietary', 'high-calcium', 'high-in-something', 'low-in-something', 'taste-mood', 'sweet', 'equipment', 'technique']","nutrition":"[1663.3, 221.0, 168.0, 66.0, 19.0, 158.0, 29.0]","n_steps":20,"steps":"['in a large bowl , mix flour , granulated sugar , baking powder and salt', 'make a well in the center of the flour mixture , and pour in milk , vanilla , egg and 2 tablespoons oil', 'mix until smooth', 'heat remaining vegetable oil to 400 degrees in a small deep fryer', 'oil is hot enough when it starts to ripple', 'be careful not to overheat', 'butter one side of each bread slice', 'sprinkle the brown sugar and cinnamon evenly onto the buttered side of the bread slices', 'make sandwiches by placing the slices of bread together with the brown sugar on the inside', 'cut the sandwiches into quarters', 'now is a good time to test the oil by dropping in a very small amount of batter', 'batter should sizzle and brown in about 20 seconds', 'oil needs to be hot enough to melt the brown sugar inside the sandwiches but not so hot that it scorches the batter', 'dip the sandwiches into the batter mixture', 'using deep fryer tongs , carefully place the battered sandwiches into the heated oil', 'be cautious when deep frying', 'try to avoid burns from splattering hot oil', 'remove sandwiches from the oil when they are golden brown', 'set aside to cool for a couple of minutes', 'sprinkle with powdered sugar and serve']","description":"my mother used to make this for us as a special treat. i don't know where she got this recipe or what it is called (hence the title). my husband loves this and encouraged me to submit it. this recipe is full of sugar and oil and is as unhealthy as a desert can be but very yummy.","ingredients":"['all-purpose flour', 'granulated sugar', 'baking powder', 'salt', 'vanilla extract', 'egg', 'milk', 'vegetable oil', 'bread', 'brown sugar', 'ground cinnamon', 'butter', 'powdered sugar']","n_ingredients":13}"""

ingredients_5 = """'bacon', 'onion', 'celery', 'carrot', 'garlic', 'butter', 'olive oil', 'lean ground beef', 'ground pork', 'beef consomme', 'dry white wine', 'crushed tomatoes', 'salt', 'black pepper', 'rubbed sage', 'oregano', 'red pepper flakes', 'nutmeg', 'milk', 'penne pasta'"""
assistant_content_5 = """{"name":"real  italian bolognese sauce","minutes":160,"tags":"['time-to-make', 'course', 'cuisine', 'preparation', 'sauces', 'condiments-etc', 'european', 'italian', 'dietary', '4-hours-or-less']","nutrition":"[1260.7, 97.0, 11.0, 71.0, 103.0, 119.0, 38.0]","n_steps":16,"steps":"['in a dutch oven or medium size pot , heat butter and olive oil over medium heat until butter begins to froth', 'add onion , celery , carrot , garlic , and bacon', 'cook until onions are translucent', 'remove bacon and remove fat', 'chop lean portions of bacon in small pieces and return to pot', 'add ground beef and ground pork , and cook until meat loses red , raw color', 'raise heat and add wine and consomme', 'cook sauce until wine and consomme are mostly evaporated', 'turn heat down to simmer and add oregano , salt , pepper , sage , red pepper flakes , and nutmeg', 'let cook for approximately 20 minutes', 'add crushed tomatoes and bring heat to a boil', 'once the mixture comes to a boil , return to simmer', 'let sauce simmer partially covered for about 2 to 4 hours , stirring occasionally to prevent sticking', 'about 5 to 10 minutes before serving , add milk', 'sauce can now be added to cooked penne pasta , spaghetti or many other pastas to your liking', 'remaining sauce may be frozen for up to two months for future use']","description":"after traveling throughout italy, savoring the fine tastes of bolognese from the many different regions, i decided to formulate my own. try it, you'll love it.","ingredients":"['bacon', 'onion', 'celery', 'carrot', 'garlic', 'butter', 'olive oil', 'lean ground beef', 'ground pork', 'beef consomme', 'dry white wine', 'crushed tomatoes', 'salt', 'black pepper', 'rubbed sage', 'oregano', 'red pepper flakes', 'nutmeg', 'milk', 'penne pasta']","n_ingredients":20}"""

ingredients_6 = """'butter', 'lemon, juice of', 'salt', 'white pepper', 'egg yolks'"""
assistant_content_6 = """{"name":"easiest ever  hollandaise sauce","minutes":25,"tags":"['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'very-low-carbs', 'sauces', 'condiments-etc', 'eggs-dairy', 'eggs', 'stove-top', 'dietary', 'low-carb', 'savory-sauces', 'low-in-something', 'equipment', 'number-of-servings']","nutrition":"[1290.4, 213.0, 4.0, 53.0, 22.0, 417.0, 1.0]","n_steps":7,"steps":"['cut the butter into several pieces and bring to room temperature', 'in the top of a double boiler , combine egg yolks , lemon juice , salt and pepper', 'add a piece of butter', 'cook , stirring steadily with a wooden spoon or wire whisk , over , but not touching , boiling water', 'when butter melts and sauce begins to thicken , add remaining butter , stirring constantly until melted', 'continue cooking as sauce thickens , about 2 more minutes', 'immediately remove from heat']","description":"the secret to this easy hollandaise sauce is in separating the egg yolks. remove all the egg whites, as they can thin the sauce. also, it is best prepared in a double boiler to prevent overheating. serve over cooked asparagus, broccoli, or broiled tomatoes.","ingredients":"['butter', 'lemon, juice of', 'salt', 'white pepper', 'egg yolks']","n_ingredients":5}"""

ingredients_7 = """'unsalted butter', 'carrot', 'onion', 'celery', 'broccoli stem', 'dried thyme', 'dried oregano', 'dried sweet basil leaves', 'dry white wine', 'chicken stock', 'worcestershire sauce', 'tabasco sauce', 'smoked chicken', 'black beans', 'broccoli floret', 'heavy cream', 'salt & fresh ground pepper', 'cornstarch'"""
assistant_content_7 = """{"name":"george s at the cove  black bean soup","minutes":90,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'soups-stews', 'beans', 'poultry', 'american', 'chicken', 'stove-top', 'dietary', 'gluten-free', 'comfort-food', 'californian', 'black-beans', 'free-of-something', 'meat', 'taste-mood', 'equipment', 'grilling', '4-hours-or-less']","nutrition":"[804.7, 108.0, 26.0, 19.0, 28.0, 214.0, 10.0]","n_steps":11,"steps":"['in 1 \/ 4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes', 'add thyme , oregano and basil', 'saute 5 minutes more', 'add wine and deglaze pan', 'add hot chicken stock and reduce by one-third', 'add worcestershire sauce , tabasco , smoked chicken , beans and broccoli florets', 'simmer 5 minutes', 'add cream , simmer 5 minutes more and season to taste', 'drop in remaining butter , piece by piece , stirring until melted and serve immediately', 'smoked chicken: on a covered grill , slightly smoke boneless chicken , cooking to medium rare', 'chef meskan uses applewood chips and does not allow the grill to become too hot']","description":"an original recipe created by chef scott meskan, george's at the cove. we enjoyed this when we visited this restaurant in la jolla, california. this recipe is requested so often, they have it printed and ready at the hostess stand. it's unbeatable at the restaurant, but i do a pretty good job at home, too, if i do say so myself!","ingredients":"['unsalted butter', 'carrot', 'onion', 'celery', 'broccoli stem', 'dried thyme', 'dried oregano', 'dried sweet basil leaves', 'dry white wine', 'chicken stock', 'worcestershire sauce', 'tabasco sauce', 'smoked chicken', 'black beans', 'broccoli floret', 'heavy cream', 'salt & fresh ground pepper', 'cornstarch']","n_ingredients":18}"""

ingredients_8 = """'veal roast', 'butter', 'oil', 'carrots', 'onions', 'parsley sprigs', 'bay leaf', 'thyme', 'salt', 'pepper', 'bacon'"""

In [16]:
# Few Shot. Model is limited a token limit of 4096

# Send request to Azure OpenAI model
response = client.chat.completions.create(
   model="gpt-35-turbo",
   temperature=0.7,
   #max_tokens=120,
   messages=[
       {"role": "system", "content": "Act as a head chef and create flavourful recipe from a list of ingredients"},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_1},
       {"role": "assistant", "content": assistant_content_1},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_2},
       {"role": "assistant", "content": assistant_content_2},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_3},
       {"role": "assistant", "content": assistant_content_3},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_4},
       {"role": "assistant", "content": assistant_content_4},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_5},
       {"role": "assistant", "content": assistant_content_5},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_6},
       {"role": "assistant", "content": assistant_content_6},
       {"role": "user", "content": "ingredients:" + "\n" + "---" + "\n" + ingredients_7},
       {"role": "assistant", "content": assistant_content_7},
       {"role": "user", "content": ingredients_8}
   ]
)

print(response.choices[0].message.content + "\n")

Summary: {"name":"classic roast veal with vegetables","minutes":180,"tags":"['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'main-dish', 'meat', 'european', 'french', 'oven', 'dietary', 'low-carb', 'low-sodium', 'low-in-something', '4-hours-or-less']","nutrition":"[675.1, 58.0, 10.0, 7.0, 19.0, 85.0, 8.0]","n_steps":12,"steps":"['preheat oven to 350 degrees', 'heat butter and oil in a large skillet', 'brown veal on all sides', 'remove veal from skillet and set aside', 'add carrots , onions , parsley , bay leaf and thyme to skillet', 'saute for 5 minutes', 'place veal on top of vegetables', 'season with salt and pepper', 'cover with bacon slices', 'cover skillet and place in preheated oven', 'roast for approximately 2 1 \/ 2 to 3 hours or until veal is tender', 'remove bay leaf before serving']","description":"this is a classic recipe for roast veal with vegetables. it is a delicious and comforting dish that is perfect for a special occasion or a comf